# Dessin selon des coordonnées géographiques

Proposition de décomposition du problème :

- reconnaître la nécessité d'un choix de projection
- choisir une projection
    - projection naïve
    - projection de Mercator
- calcul des coordonnées extrêmes (*bounding box*)
- calcul de l'échelle (horizontale / verticale)

## Projection géographique

Problème : les coordonnées géographiques [WGS84](https://fr.wikipedia.org/wiki/WGS_84) sont exprimées en *degrés* (coordonnées sphériques). Pour les représenter sur une surface plane, il faut associer à chaque couple (longitude, latitude) un couple (abscisse, ordonnée) sur un plan. On appelle cela une [projection cartographique](https://fr.wikipedia.org/wiki/Projection_cartographique). Il existe de nombreuses manières de faire, qui ont toutes différents avantages et inconvénients (déformations aux pôles, non-conservation des aires...). La plus courante est la [*projection de Mercator*](https://fr.wikipedia.org/wiki/Projection_de_Mercator).

In [3]:
# quelques points du contour de la Seine-et-Marne
points = [(2.3923284961351237, 48.335929161584076), (2.5796456483373684, 48.72254039779301), (2.6241630097148376, 49.10056541579139), (3.097802797538821, 49.11080329041811), (3.26191242353348, 48.94707797669175), (3.4474402215212865, 48.848328384354076), (3.4613763255130037, 48.634670779600604), (3.4003188237982123, 48.45958741212868), (3.0995219034980406, 48.3529392805501), (2.8310023496715675, 48.13421017601398), (2.5384374109720103, 48.14073211263637), (2.419849013684824, 48.28321626105787)]
bbox = [2.3923284961351237, 48.12014561527111, 3.559220826259302, 49.11789167125887]

In [1]:
from math import log, tan, pi, radians, degrees

def mercator_map(point):
    """Spherical Mercator map of p."""
    long, lat = point
    x = long
    y = degrees(log(tan(radians(lat) / 2 + pi / 4)))
    return x, y

In [4]:
points_mercator = [mercator_map(p) for p in points]
points_mercator

[(2.3923284961351237, 55.36249318140423),
 (2.5796456483373684, 55.94629229454141),
 (2.6241630097148376, 56.5214811455119),
 (3.097802797538821, 56.5371194671355),
 (3.26191242353348, 56.28741521676643),
 (3.4474402215212865, 56.13720405535307),
 (3.4613763255130037, 55.813213353069195),
 (3.4003188237982123, 55.54873810448331),
 (3.0995219034980406, 55.38808573469113),
 (2.8310023496715675, 55.05964585646803),
 (2.5384374109720103, 55.06941880810703),
 (2.419849013684824, 55.28323823748444)]

In [8]:
from fltk import cree_fenetre, polygone, attend_ev, ferme_fenetre

cree_fenetre(500, 500)
# Dessiner le polygone (mais comment ?)
# polygone(points_mercator)
attend_ev()
ferme_fenetre()

In [ ]:
import fltk

help(fltk.polygone)

## Calcul des coordonnées à l'écran

Contrainte : il faut garder l'aspect (ratio longueur / largeur)

Stratégie :
- Calculer d'abord le bounding box (point nord-ouest et point sud-est) en coordonnées géographiques
- Chercher le plus grand facteur de zoom possible pour faire tenir la bounding box sur la zone de dessin

Maintenant, supposons que 
- Le point nord-ouest du bounding box est $(x_1, y_1)$, et celui sud-est est $(x_2, y_2)$.
- La zone de dessin est un rectangle de largeur $W$, de hauteur $H$, de coin supérieur gauche $(X_O, Y_O)$ et de coin inférieur droit $(X_O + W, Y_O + H)$.

(Convention : coordonnées géographiques en minuscules, coordonnées écran (pixels) en majuscules.)

### Comment convertir un point géographique $(x, y)$ en un point $(X, Y)$ sur la fenêtre ?


Cela doit être une transformation linéaire (sans déformation) et respectant les proportions (même facteur en abscisses et en ordonnées).  
⚠️⚠️⚠️ **Attention** : les ordonnées sont inversées sur la fenêtre !
$$
    \begin{cases}
    X = ax + B\\
    Y = a(- y) + C.
    \end{cases}
$$

**Contrainte :** Les points correspondant aux coins nord-ouest et sud-est doivent tenir tous les deux dans la zone de dessin :
$$
    \begin{cases}
    X_O \leq ax_1 + B \leq ax_2 + B \leq X_O + W\\
    Y_O \leq - ay_1 + C \leq -ay_2 + C \leq Y_O + H
    \end{cases}
$$

Il ne reste plus qu'à résoudre les inéquations pour encadrer $a$, $B$ et $C$ !

#### Encadrement du facteur d'échelle $a$


On a d'une part, pour les abscisses :
$$
X_O - B \leq ax_1 \leq ax_2 \leq X_O + W - B
$$

En réarrangeant on obtient :
$$
\begin{aligned}
ax_2 - ax_1 & \leq (X_O + W - B) - (X_O - B)\\
a(x_2 - x_1) & \leq W\\
a & \leq W / (x_2 - x_1)
\end{aligned}
$$

Pour les ordonnées, on a :
$$
Y_O - C \leq -ay_1 \leq -ay_2 \leq Y_O + H - C
$$

Soit, en réarrangeant :
$$
\begin{aligned}
a(y_1 - y_2) & \leq (Y_O + H - C) - (Y_O - C)\\
a(y_1 - y_2) & \leq H\\
a & \leq H / (y_1 - y_2)
\end{aligned}
$$


On peut donc choisir par exemple $$a = \min \left\{W / (x_2 - x_1), H / (y_1 - y_2)\right\}$$ qui correspond à un zoom maximal.

#### Encadrement des décalages $B$ et $C$


On peut encadrer $B$ et $C$ ainsi :

$$
    \begin{cases}
    X_O - ax_1 \leq B \leq X_O + W - ax_2\\
    Y_O + ay_1 \leq C \leq Y_O + H + ay_2
    \end{cases}
$$

On pourra choisir n'importe quelles valeurs qui conviennent pour placer le dessin où l'on veut (contre l'un des bords, centré...). 


Notons que si $a = W / (x_2 - x_1)$ (dessin ajusté en largeur) alors 
$$
\begin{aligned}
X_O - a x_1 \leq B & \leq X_O + W - W x_2 / (x_2 - x_1)\\
                   & \leq X_O + (W (x_2 - x_1) - W x_2) / (x_2 - x_1)\\
                   & \leq X_O - W x_1 / (x_2 - x_1)\\
                   & \leq X_O - a x_1
\end{aligned}
$$
autrement dit $B$ est forcément égal à $0_x - a x_1$.


De même si $a = H / (y_1 - y_2)$ (dessin ajusté en hauteur) alors 
$$
\begin{aligned}
Y_O + a y_1 \leq C & \leq Y_O + H + H y_2 / (y_1 - y_2)\\
                   & \leq Y_O + (H (y_1 - y_2) + H y_2) / (y_1 - y_2)\\
                   & \leq Y_O + H y_1 / (y_1 - y_2)\\
                   & \leq Y_O + a y_1
\end{aligned}
$$
autrement dit $C$ est forcément égal à $Y_O + a y_1$.

### Implémentation en Python

In [ ]:
def frame_map(point, origin, scale):
    """Screen map of point relative to (upper-left) origin and scale."""
    ... # à finir

In [ ]:
def compute_frame(bbox, max_dim):
     # à finir
    long_min, lat_min, long_max, lat_max = bbox

    # coordonnées des coins
    xmin, ymax = ...
    xmax, ymin = ...

    # amplitude en abscisse et en ordonnée
    dx, dy = ...

    # échelle
    scale = ...

    return (xmin, ymax), (xmax, ymin), (dx, dy), scale